https://github.com/TendTo/Quantum-random-walk-simulation/blob/main/QuantumWalk.ipynb

In [ ]:
from pygments.lexers import graphviz
import numpy as np
import networkx as nx
from networkx import hypercube_graph
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
from typing import Callable
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info import Operator
from qiskit.visualization import circuit_drawer, plot_histogram, plot_bloch_vector
from qiskit_aer import AerSimulator
backend = AerSimulator()
from numpy import pi
import pylatexenc
import graphviz
import pygraphviz
DIMENSIONS = 2


Quantum walk on a hypercube
The quantum random walk can be used to walk on the vertices of a hypercube.

Hypercube
A hypercube with degree 
 is a graph with 
 vertices, where each vertex is connected to all the other vertices that differ by a single bit.

In [ ]:
n = DIMENSIONS # Number of qubits

G = hypercube_graph(n)  # Create hypercube graph
# Draw graph
nx.draw(
    G,
    with_labels=True,
    pos=graphviz_layout(G, prog="dot"),
    node_color="#dddddd",
    node_size=2500,
)
plt.show()

Quantum registers
A circuit implementing a quantum coined needs:

 qubits to represent the coin
the coin will yield one of the 
 possible outcomes, indicating which qubit to invert
 qubits to represent the walker position
the numbers are represented in binary form, with the most significant bit at the top
 classical bits to store the measurement of the walker position

In [ ]:

n = DIMENSIONS  # Dimension of the hypercube


def graph_walk_circuit(
    n: int,
) -> tuple[QuantumCircuit, QuantumRegister, QuantumRegister, ClassicalRegister]:
    """Create a quantum circuit for the graph walk.

    Args:
        n: number of qubits used to represent the position of the walker

    Returns:
        quantum circuit for the graph walk, walker register, coin register, and output register
    """
    walker_r = QuantumRegister(2**n, name="w")
    coin_r = QuantumRegister(n, name="c")
    classic_r = ClassicalRegister(2**n, name="output")
    qc = QuantumCircuit(walker_r, coin_r, classic_r)
    return qc, walker_r, coin_r, classic_r


qc, _, _, _ = graph_walk_circuit(n)
qc.draw(output="mpl", initial_state=True)

Operators
Coin operator
In this case, the coin operator is the Grover coin, 
. It is defined as:

In [ ]:
n = DIMENSIONS  # Dimension of the hypercube

def grover_coin(coin_r: QuantumRegister) -> Operator:
    matrix_size = 2 ** len(coin_r)
    grover_matrix = np.full((matrix_size, matrix_size), 2 / matrix_size) - np.eye(
        matrix_size
    )
    return Operator(grover_matrix)


coin_r = QuantumRegister(n, name="c")
qc = QuantumCircuit(coin_r)
gc = grover_coin(coin_r)
qc.unitary(gc, coin_r[:], label="Grover Coin")
qc.draw(output="mpl", initial_state=True)

Shift operator
Let 
 be the vector with a single 1 at position 
 and 0s everywhere else. The shift operator 
 is defined as:

In [ ]:
n = DIMENSIONS  # Dimension of the hypercube


def shift_operator(
    walker_r: QuantumRegister, coin_r: QuantumRegister
) -> QuantumCircuit:
    """Create a quantum circuit for the shift operator.

    Args:
        walker_r: the quantum register containing the walker's position qubits
        coin_r: the quantum register containing the coin qubit

    Returns:
        quantum circuit for the shift operator
    """
    qc = QuantumCircuit(walker_r, coin_r)
    for i in reversed(range(len(walker_r))):
        qc.mcx(coin_r, walker_r[i])
        qc.x(coin_r[-1])
        for j in range(1, len(coin_r)):
            if i & ((1 << j) - 1) == 0:
                qc.x(coin_r[-(j + 1)])
    return qc


qc, walker_r, coin_r, _ = graph_walk_circuit(n)
qc = shift_operator(walker_r, coin_r)
qc.draw(output="mpl", initial_state=True)

Quantum walk operator
The quantum walk operator is the composition of the coin operator and the shift operator:

In [ ]:
n = DIMENSIONS # Dimension of the hypercube


def graph_walk_step(
    walker_r: QuantumRegister, coin_r: QuantumRegister
) -> QuantumCircuit:
    """Create a quantum circuit for one step of the graph walk.

    Args:
        walker_r: the quantum register containing the walker's position qubits
        coin_r: the quantum register containing the coin qubit

    Returns:
        quantum circuit for one step of the graph walk
    """
    shift = shift_operator(walker_r, coin_r)
    gc = grover_coin(coin_r)
    walk_step = QuantumCircuit(walker_r, coin_r)
    walk_step.unitary(gc, coin_r, label="Grover Coin")
    walk_step.compose(shift, inplace=True)
    return walk_step


walk_step = graph_walk_step(walker_r, coin_r)
walk_step.draw(output="mpl", initial_state=True)

Circuit
With all the operators defined, the circuit can be built by iterating the single step operator as many times as needed. Finally, the walker position is measured and the result is stored in the classical bits.

In [ ]:
n = DIMENSIONS  # Dimension of the hypercube
n_steps = 3  # Number of steps of the quantum walk


def graph_walk(
    n: int,
    n_steps: int,
) -> QuantumCircuit:
    """Create a quantum circuit for the quantum walk over a graph.

    Args:
        n: dimension of the hypercube
        n_steps: number of steps of the quantum walk
        initial_position: initial position of the walker
        initial_coin_value: initial value of the coin

    Returns:
        quantum circuit for the quantum walk over a graph
    """
    qc, walker_r, coin_r, classic_r = graph_walk_circuit(n)
    for i in range(n_steps):
        step_gate = graph_walk_step(walker_r, coin_r).to_gate(label=f"Step #{i + 1}")
        qc.append(step_gate, walker_r[:] + coin_r[:])
    qc.barrier()
    qc.measure(walker_r, reversed(classic_r))
    return qc


qc = graph_walk(n, n_steps)
qc.draw(output="mpl", initial_state=True)

Results of the simulation
The selection of the coin operator determines the behavior of the quantum walk. The Grover coin is used to produce a biased walk.

In [ ]:
n = DIMENSIONS # Dimension of the hypercube
n_steps = 30  # Number of steps of the quantum walk
n_runs = 5000  # Number of runs of the quantum circuit

qc = graph_walk(n, n_steps)

def plot_results(qc: QuantumCircuit, n_runs: int, title: str):
    """Plot the results of the quantum walk.

    Args:
        qc: quantum circuit for the quantum walk
        n_runs: number of runs of the quantum circuit

    Returns:
        histogram of the results of the quantum walk
    """
    simulator = AerSimulator() 
    transpiled_qc = transpile(qc, simulator)
    results = simulator.run(transpiled_qc, shots=n_runs).result()
    counts = results.get_counts(transpiled_qc)
    counts = {int(k, 2): v for k, v in counts.items()}
    return plot_histogram(counts, title=title)


plot_results(qc, n_runs, "Quantum Walk on a Hypercube")